In [1]:
pip install weaviate-client -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
WEAVIATE_KEY=""
OPENAI_APIKEY=""

In [22]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key=WEAVIATE_KEY)

client = weaviate.Client(
      url="https://sm-sandbox-cx8g2wxo.weaviate.network",
      auth_client_secret=auth_config,
      additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_APIKEY # Replace with your inference API key
    }
)

client

In [18]:
client.schema.delete_class("Question")

In [19]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

In [20]:
# get the schema
schema = client.schema.get()

# print the schema
print(json.dumps(schema, indent=4))

{
    "classes": [
        {
            "class": "Question",
            "invertedIndexConfig": {
                "bm25": {
                    "b": 0.75,
                    "k1": 1.2
                },
                "cleanupIntervalSeconds": 60,
                "stopwords": {
                    "additions": null,
                    "preset": "en",
                    "removals": null
                }
            },
            "moduleConfig": {
                "generative-openai": {},
                "text2vec-openai": {
                    "model": "ada",
                    "modelVersion": "002",
                    "type": "text",
                    "vectorizeClassName": true
                }
            },
            "multiTenancyConfig": {
                "enabled": false
            },
            "properties": [],
            "replicationConfig": {
                "factor": 1
            },
            "shardingConfig": {
                "virtualPerPhysical": 128,
   

In [23]:
import requests, json
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(data):  # Batch import data
        print(f"importing question: {i+1}")
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [24]:
import requests
fname = "jeopardy_tiny_with_vectors_all-OpenAI-ada-002.json"  # This file includes pre-generated vectors
url = f'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/{fname}'
resp = requests.get(url)
data = json.loads(resp.text)  # Load data

client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Configure a batch process
    for i, d in enumerate(data):  # Batch import all Questions
        print(f"importing question: {i+1}")
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Question",
            vector=d["vector"]  # Add custom vector
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [27]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["biology"]})
    .with_where({
        "path": ["category"],
        "operator": "Equal",
        "valueText": "ANIMALS"
    })
    .with_limit(2)
    .do()
)
response

{'data': {'Get': {'Question': [{'answer': 'Elephant',
     'category': 'ANIMALS',
     'question': "It's the only living mammal in the order Proboseidea"},
    {'answer': 'the nose or snout',
     'category': 'ANIMALS',
     'question': 'The gavial looks very much like a crocodile except for this bodily feature'}]}}}

In [29]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["biology"]})
    .with_where({
        "path": ["category"],
        "operator": "Equal",
        "valueText": "ANIMALS"
    })
    .with_generate(single_prompt="Explain {answer} as you might to a five-year-old.")
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "_additional": {
                        "generate": {
                            "error": null,
                            "singleResult": "An elephant is a really big animal with a long trunk, big ears, and a strong body. They are usually gray in color. Elephants are very smart and friendly. They live in places called forests and grasslands. They eat lots of plants and fruits. They use their long trunk to grab food and drink water. Elephants also use their trunk to say hello to other elephants by touching them gently. They have big ears that help them hear really well. Elephants are very strong and can carry heavy things with their trunk. They are also great swimmers and love to play in the water. Elephants are loved by many people because they are so amazing and special!"
                        }
                    },
                    "answer": "Elephant",
                    "cat

In [32]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["biology"]})
    .with_generate(grouped_task="Write a tweet with emojis about these facts.")
    .with_limit(2)
    .do()
)

print(response["data"]["Get"]["Question"][0]["_additional"]["generate"]["groupedResult"])
print(json.dumps(response, indent=4))

"🧬 Did you know? In 1953, Watson & Crick 🧪 built a model of the molecular structure of DNA, the gene-carrying substance! 🧬 #ScienceFacts"
{
    "data": {
        "Get": {
            "Question": [
                {
                    "_additional": {
                        "generate": {
                            "error": null,
                            "groupedResult": "\"\ud83e\uddec Did you know? In 1953, Watson & Crick \ud83e\uddea built a model of the molecular structure of DNA, the gene-carrying substance! \ud83e\uddec #ScienceFacts\""
                        }
                    },
                    "answer": "DNA",
                    "category": "SCIENCE",
                    "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
                },
                {
                    "_additional": {
                        "generate": null
                    },
                    "answer": "DNA",
        

In [35]:
q = client.query\
    .get("Question", ["question", "answer", "category"])\
    .with_near_text({"concepts": ["biology"]})\
    .with_where({\
        "path": ["category"],\
        "operator": "Equal",\
        "valueText": "ANIMALS"\
    })\
    .with_generate(single_prompt="Explain {answer} as you might to a five-year-old.")\
    .with_limit(2)

q.build()


'{Get{Question(where: {path: ["category"] operator: Equal valueText: "ANIMALS"} limit: 2 nearText: {concepts: ["biology"]} ){question answer category _additional {generate(singleResult:{prompt:"Explain {answer} as you might to a five-year-old."}){error singleResult} }}}}'